In [1]:
import numpy as np
import pandas as pd

## Dataset:

In [101]:
colnames = ['eng', 'ni1', 'ni2', 'vn']
path = '/content/drive/MyDrive/Fall 2022 - Assignment/Software Construction/Project/full_data/main_df.csv'
df = pd.read_csv(path, header=0)
df.head()
print(df.shape)

(31481, 5)


In [102]:
df = df.drop('Unnamed: 0', axis=1)

In [103]:
df = df.drop(['ni1', 'ni2'], axis=1)

### Droping NAs

In [104]:
missing = df[df.isna().any(axis=1)]
#missing.to_csv('/content/drive/MyDrive/Fall 2022 - Assignment/Software Construction/Project/full_data/missing.csv')
missing.shape 

(2231, 2)

In [105]:
# Dropping NAs
df = df.dropna(axis='index', how='any')
print(df.shape)

(29250, 2)


### Split cases with ```/``` into multiple rows
Example: ```abulia (n) / abulic (a)	``` becomes 2 rows ```abulia (n)``` and ```abulic (a)```

In [106]:
df_split = df.loc[df.eng.str.contains('/'), :].copy() # create a new df of only rows with '/'

df = df.loc[~df.eng.str.contains('/'), :].copy() # remove them for now

print(df_split.shape)
print(df.shape)

(1292, 2)
(27958, 2)


In [107]:
# SOURE: https://stackoverflow.com/questions/58523316/split-rows-in-pandas-dataframe

# for index, row in df.iterrows():
#     split = row['Shape'].split(' / ')
#     for shape in split:
#         new_df = new_df.append(pd.DataFrame({'Color':[row['Color']], 'Shape':[shape], 'Price':[row['Price']]}))

# new_df = new_df.reset_index(drop=True)

df_split['eng'] = df_split.eng.str.split(" / ")
df_split = df_split.explode("eng").reset_index(drop=True)
df_split.head(2)
print(df_split.shape)

(2575, 2)


In [108]:
# Append the splitted '/' cases back to the orginal data

df = df.append(df_split, ignore_index=True)
df.shape

(30533, 2)

### Split cases with ```,``` into multiple rows
Example: ```abiogenetic, abiogenous (a)``` becomes 2 rows : ```abiogenetic (a)``` and ```abiogenous (a)```

In [109]:
df_split_comma = df.loc[df.eng.str.contains(', '), :].copy() # extract rows with ', '
df = df.loc[~df.eng.str.contains(', '), :].copy() # temporarily remove these rows
print(df.shape)
print(df_split_comma.shape)

(29469, 2)
(1064, 2)


In [110]:
df_split_comma['eng'] = df_split_comma.eng.str.split(", ")
df_split_comma = df_split_comma.explode("eng").reset_index(drop=True)
df_split_comma.head(2)

,eng,vn
0,arthropneumography,chụp khớp bơm khí
1,arthropneumoradiography (n),chụp khớp bơm khí


**Note**: Later on when adding word types (a, n, v...) we will use ```fillna('bfill')``` to account for those without a type
Example: ```-aberration``` comes right before ```aberratio (n)``` so will be backfilled with ```(n)```

In [113]:
df = df.append(df_split_comma, ignore_index=True)
print(df.shape)

(31661, 2)


### Extract word types (n a adj v...)

In [114]:
df['word_type'] = df.eng
df.head(3)

,eng,vn,word_type
0,arthropathology (n),bệnh học khớp,arthropathology (n)
1,-arthropathy (n),bệnh khớp,-arthropathy (n)
2,arthrophyma (n),sưng khớp,arthrophyma (n)


#### Remove cases with special ```ñn```

In [115]:
def remove_special(entry):
  '''
  For words that contain 'ñn', remove the part that comes after this symbol
  '''
  if 'ñn' in entry:
    return entry.split('ñn')[0]
  else:
    return entry
# lambda x: x.split('ñn')[0] if 'ñn' in x else x

#### Extract word types

In [116]:
def get_word_type(entry):
  '''
  Works for easy cases. Extract the word type (n, a, v, ...) inside the brackets '()'
  '''
  entry = entry.rstrip()
  if '(' in entry:
    if len(entry.split('(')) == 2:
      return entry.split('(')[1][:-1]

    else:
      return "More than 1 ()'s"

  else:
    return np.nan

In [117]:
df.word_type = df.word_type.apply(remove_special)
df.word_type = df.word_type.apply(get_word_type)

In [118]:
df_word_type_na = df.loc[df.word_type.isna(), :].copy() # extract all words with NA
df = df.loc[~df.word_type.isna(), :].copy() # temporarily remove these words

print(df.shape)
print(df_word_type_na.shape)

(30368, 3)
(1293, 3)


#### Fill NAs
Before, there were a few rows resulted from splitting ```', '``` that don't have a word type ```(n)``` afterwards. We fix this by matching them with the word type of the **"true"** words - which are the final words in a ```', '``` that contain the word type.

Example: ```-aberration, aberratio (n)``` => ```abrratio``` is the **true** word, containing the type ```(n)```.

In [123]:
def get_word_type_for_na(entry):
  if entry in df_split_comma.eng.values:
    curr_idx = np.where(df_split_comma.eng.values == entry)[0][0] # index of the entry

    move = 1
    while df_split_comma.eng.values[curr_idx + move][-1] != ')': # sometimes more than 1 word before "true" word
      move += 1 # increase this until we find the "true" label.

    type_idx = curr_idx + move # "true" word with word type
    type_value = df_split_comma.eng.values[type_idx]

    try:
      return  df.loc[df.eng==type_value, 'word_type'].values[0]
    except IndexError:
      return np.nan
  
  else:
    return np.nan

In [124]:
df_word_type_na.word_type = df_word_type_na.eng
df_word_type_na.word_type = df_word_type_na.word_type.apply(get_word_type_for_na)
df_word_type_na.tail(10)

,eng,vn,word_type
31641,toxicopectic,"trung hòa chất độc, cố định chất độc",a
31643,-trigonocephaly,đầu hình tam giác (dị hình),n
31645,-triphalangism,ba đốt ngón cái (tật),n
31647,-trisomy,thể ba,n
31649,-tritanopia,mù màu lam (ch),n
31651,trochanteric,mấu chuyển,a
31653,urologic,"niệu học, khoa tiết niệu",a
31655,urticarial,mề đay,a
31657,-vagotonia,"cường thần kinh phế vị, cường thần kinh đối gi...",n
31659,vulval,âm hộ,a


In [129]:
df = df.append(df_word_type_na, ignore_index=True)
df.shape

(31661, 3)

### Too edgy edge cases
The rest are edge cases with no rules or standards for now.

In [130]:
df.word_type.value_counts()

n                                                   21707
a                                                    7803
More than 1 ()'s                                      479
a,n                                                   377
v                                                     256
                                                    ...  
n) coi coù ñuùng hypnosie cuûa Faùp khoâng??????        1
human immunodeficiency virus.                           1
diffraction                                             1
retrolental                                             1
a)///////////////////                                   1
Name: word_type, Length: 143, dtype: int64

In [132]:
df[df.word_type=='diffraction']

,eng,vn,word_type
8787,grating (diffraction),mạng nhiễu xạ,diffraction


In [133]:
edge = df_word_type_na.loc[df_word_type_na.word_type.isna(), :].copy()
df_word_type_na = df_word_type_na.loc[~df_word_type_na.word_type.isna(), :].copy()

df = df.append(df_word_type_na, ignore_index=True)

In [134]:
more_edge = df.loc[df.word_type=="More than 1 ()'s", :].copy()
df = df.loc[df.word_type!="More than 1 ()'s", :].copy()

edge = edge.append(more_edge, ignore_index=True)

In [135]:
edge

,eng,vn,word_type
0,-articulo-mortis,"lúc hấp hối, lúc lâm chung",NaN
1,-ascorbic acid,acid ascorbic,NaN
2,-atrioventricularis communis,ống nhĩ thất chung,NaN
3,-axis cylinder,"sợi trục, trụ trục",NaN
4,anat.,giải phẫu học,NaN
...,...,...,...
676,kine-,"động, vận động",More than 1 ()'s
677,leptocephalic,đầu hẹp (th),More than 1 ()'s
678,monorchidism,một tinh hoàn (quái tượng ),More than 1 ()'s
679,thele-,núm vú,More than 1 ()'s


In [136]:
# RUN ONCE
edge.to_csv('/content/drive/MyDrive/Fall 2022 - Assignment/Software Construction/Project/full_data/edge_cur.csv')

#### Remove the word types in ```eng``` column

In [137]:
df.head()

,eng,vn,word_type
0,arthropathology (n),bệnh học khớp,n
1,-arthropathy (n),bệnh khớp,n
2,arthrophyma (n),sưng khớp,n
3,arthrophyte (n),dị vật khớp,n
4,arthropod (n),"chân khớp (động vật), tiết chi động vật",n


In [138]:
# Idea is to split on "(", and keep only the [0] part
def remove_word_type(entry):
  if "(" in entry:
    keep = entry.split('(')[0].rstrip()
    return keep
  else:
    return entry

df.eng = df.eng.apply(remove_word_type)

### Add new ```word_type_vn``` column


In [139]:
df.word_type.value_counts()

n                                                   22397
a                                                    8164
a,n                                                   389
v                                                     261
pref.                                                 125
                                                    ...  
n) coi coù ñuùng hypnosie cuûa Faùp khoâng??????        1
human immunodeficiency virus.                           1
diffraction                                             1
retrolental                                             1
a)///////////////////                                   1
Name: word_type, Length: 142, dtype: int64

In [140]:
df['word_type_vn'] = df.word_type


In [141]:
df.word_type_vn.value_counts()

n                                                   22397
a                                                    8164
a,n                                                   389
v                                                     261
pref.                                                 125
                                                    ...  
n) coi coù ñuùng hypnosie cuûa Faùp khoâng??????        1
human immunodeficiency virus.                           1
diffraction                                             1
retrolental                                             1
a)///////////////////                                   1
Name: word_type_vn, Length: 142, dtype: int64

In [152]:
'pref' in 'pref.)'

True

In [148]:
df.word_type_vn.value_counts().head(45)

n                             22397
a                              8164
a,n                             389
v                               261
pref.                           125
a.n                              80
F                                75
suff.                            73
L                                60
pref.)                           34
m                                34
ad                               31
 n                               21
f                                20
disorder                         19
v,n                              19
o)                               18
suff.)                           13
n,v                              13
pref                             13
suff.)                           12
                                 12
pref.)                           10
ad.                              10
n)                                7
pref)                             6
G                                 4
o)-                         

In [156]:
def get_word_type_vn(entry):
  entry = str(entry)
  if entry == 'n':
    return 'danh từ'
  if entry == 'a':
    return 'tính từ'
  if entry == 'v':
    return 'động từ'
  if 'pref' in entry: # prefix
    return 'tiền tố'
  if 'suff' in entry:
    return 'hậu tố'
  else:
    if 'a' in entry and len(entry) <= 3:
      return 'tính từ'
    if 'n' in entry and len(entry) <= 3:
      return 'danh từ'
    if 'v' in entry and len(entry) <= 3:
      return 'động từ'
  return entry

In [157]:
df.head()

,eng,vn,word_type,word_type_vn
0,arthropathology,bệnh học khớp,n,n
1,-arthropathy,bệnh khớp,n,n
2,arthrophyma,sưng khớp,n,n
3,arthrophyte,dị vật khớp,n,n
4,arthropod,"chân khớp (động vật), tiết chi động vật",n,n


In [158]:
df.word_type_vn = df.word_type_vn.apply(get_word_type_vn)

In [159]:
df.head(10)

,eng,vn,word_type,word_type_vn
0,arthropathology,bệnh học khớp,n,danh từ
1,-arthropathy,bệnh khớp,n,danh từ
2,arthrophyma,sưng khớp,n,danh từ
3,arthrophyte,dị vật khớp,n,danh từ
4,arthropod,"chân khớp (động vật), tiết chi động vật",n,danh từ
5,-Arthropoda,chân khớp (ngành),n,danh từ
6,-arthropyosis,sinh mủ khớp (sự),n,danh từ
7,arthrorisis,hạn chế khớp (thth),n,danh từ
8,arthroscintigram,nhấp nháy đồ khớp,n,danh từ
9,arthroscintigraphy,chụp nhấp nháy khớp (thuật),n,danh từ


### Remove ```'-'``` before words

In [160]:
df.eng = df.eng.apply(lambda x: x.lstrip('-'))

### Final version

In [161]:
df.head(4)

,eng,vn,word_type,word_type_vn
0,arthropathology,bệnh học khớp,n,danh từ
1,arthropathy,bệnh khớp,n,danh từ
2,arthrophyma,sưng khớp,n,danh từ
3,arthrophyte,dị vật khớp,n,danh từ


In [162]:
df.isna().any()

eng             False
vn              False
word_type        True
word_type_vn    False
dtype: bool

In [163]:
df.shape

(32273, 4)

In [166]:
# RUN ONCE

df.to_csv('/content/drive/MyDrive/Fall 2022 - Assignment/Software Construction/Project/full_data/final_cur.csv')

In [165]:
df

,eng,vn,word_type,word_type_vn
0,arthropathology,bệnh học khớp,n,danh từ
1,arthropathy,bệnh khớp,n,danh từ
2,arthrophyma,sưng khớp,n,danh từ
3,arthrophyte,dị vật khớp,n,danh từ
4,arthropod,"chân khớp (động vật), tiết chi động vật",n,danh từ
...,...,...,...,...
32759,trochanteric,mấu chuyển,a,tính từ
32760,urologic,"niệu học, khoa tiết niệu",a,tính từ
32761,urticarial,mề đay,a,tính từ
32762,vagotonia,"cường thần kinh phế vị, cường thần kinh đối gi...",n,danh từ
